In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Sequential

import numpy as np
import matplotlib.pyplot as plt

import os
import cv2
from keras.preprocessing.image import img_to_array

# x is noisy data and y is clean data
SIZE = 2048

from tqdm import tqdm
folder = '../input/bad-img'
files=os.listdir(folder)
print(files)
sortedfiles=sorted(files)
print(sortedfiles)
noisy_data=[]
for filename in sortedfiles:
    img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_COLOR)
    img=cv2.resize(img,(SIZE, SIZE))
    noisy_data.append(img_to_array(img))

folder1 = '../input/good-img'
files1=os.listdir(folder1)
print(files1)
sortedfiles1=sorted(files1)
print(sortedfiles1)
clean_data=[]
for filename in sortedfiles1:
    img = cv2.imread(os.path.join(folder1, filename), cv2.IMREAD_COLOR)
    img=cv2.resize(img,(SIZE, SIZE))
    clean_data.append(img_to_array(img))

noisy_train = np.reshape(noisy_data, (len(noisy_data), SIZE, SIZE, 3))
noisy_train = noisy_train.astype('float32') / 255.

clean_train = np.reshape(clean_data, (len(clean_data), SIZE, SIZE, 3))
clean_train = clean_train.astype('float32') / 255.


#Displaying images with noise
plt.figure(figsize=(10, 2))
for i in range(1,4):
    ax = plt.subplot(1, 4, i)
    plt.imshow(cv2.cvtColor(noisy_train[i].reshape(SIZE, SIZE,3), cv2.COLOR_BGR2RGB))
plt.show()

#Displaying clean images
plt.figure(figsize=(10, 2))
for i in range(1,4):
    ax = plt.subplot(1, 4, i)
    plt.imshow(cv2.cvtColor(clean_train[i].reshape(SIZE, SIZE,3), cv2.COLOR_BGR2RGB))
plt.show()



model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(SIZE, SIZE, 3)))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
 

model.add(MaxPooling2D((2, 2), padding='same'))
     
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(3, (3, 3), activation='relu', padding='same'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

model.summary()

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(noisy_train, clean_train, 
                                                    test_size = 0.05, random_state = 0)


model.fit(x_train, y_train, epochs=60, batch_size=1, shuffle=True, verbose =1,
          validation_split = 0.3)


print("Test_Accuracy: {:.2f}%".format(model.evaluate(np.array(x_test), np.array(y_test))[1]*100))


#model.save('denoising_autoencoder.model')

no_noise_img = model.predict(x_test)

#Displaying images with noise
plt.figure(figsize=(10, 2))
for i in range(1,2):
    ax = plt.subplot(1, 4, i)
    img0=cv2.cvtColor(x_test[i-1].reshape(SIZE, SIZE,3), cv2.COLOR_BGR2RGB)
    #plt.imsave('./noised_image.jpg', img0)
    plt.imshow(cv2.cvtColor(x_test[i-1].reshape(SIZE, SIZE,3), cv2.COLOR_BGR2RGB))
plt.show()

#Displaying clean images
plt.figure(figsize=(10, 2))
for i in range(1,2):
    ax = plt.subplot(1, 4, i)
    img1=cv2.cvtColor(no_noise_img[i-1].reshape(SIZE, SIZE,3), cv2.COLOR_BGR2RGB)
    plt.imsave('./denoised_image.jpg', img1)
    #plt.imshow(cv2.cvtColor(no_noise_img[i-1].reshape(SIZE, SIZE,3), cv2.COLOR_BGR2RGB))
    plt.imshow(cv2.cvtColor(no_noise_img[i-1].reshape(SIZE, SIZE,3), cv2.COLOR_BGR2RGB))
plt.show()
